In [2]:
import os
from torch.utils.data import Dataset
import torch
import nibabel as nib
import numpy as np
from PIL import Image
from torch.autograd import Variable

device = torch.device('cuda')

def make_one_hot(labels, device, C=2):
    '''
    Converts integer labels to one-hot encoding for semantic segmentation tasks.
 
    Parameters
    ----------
    labels : torch.autograd.Variable of torch.cuda.LongTensor
        Shape: N x 1 x H x W, where N is the batch size.
        Each value is an integer representing correct classification.
    C : integer
        Number of classes in labels.
 
    Returns
    -------
    target : torch.autograd.Variable of torch.cuda.FloatTensor
        Shape: N x C x H x W, where C is the class number. One-hot encoded.
    '''
    # Ensure labels are of type LongTensor
    labels = labels.long().to(device)
   
    # Create a zero-initialized one-hot tensor with the appropriate dimensions
    one_hot = torch.FloatTensor(C, labels.size(1), labels.size(2), labels.size(3)).zero_().to(device)
   
    # Use scatter_ to set the corresponding class index to 1 for each pixel
    target = one_hot.scatter_(0, labels.data,1)
   
    # Convert the result to a torch.autograd.Variable
    target = Variable(target)
       
    return target

# Define a function to read NIfTI image from a given path
def readNifti_img(path):
    # Load the NIfTI image and normalize pixel values
    # print(path)
    img_ = nib.load(path).get_fdata()
    img_ = img_ / img_.max()
    # Convert to torch tensor and add channel dimension
    img_ = torch.tensor(img_, dtype=torch.float32).unsqueeze(0)
    return img_

# Define a function to read NIfTI mask from a given path
def readNifti_mask(path):
    # Load the NIfTI mask and convert to torch tensor with channel dimension
    # print(path)
    mask_ = nib.load(path).get_fdata()
    mask_ = torch.tensor(mask_, dtype=torch.float32).unsqueeze(0)
    #mask_ = make_one_hot(mask_,torch.device('cuda'),3)
    #Convert to categorical to be [1,3,112,112,112]
    return mask_

class NiftiDataset(Dataset):
   def __init__(self, image_paths, mask_paths, transform=None):
       # Initialize dataset with image and mask paths, fixed, moving, and transform
       self.image_paths = image_paths
       self.mask_paths = mask_paths
       self.transform = transform


   def __len__(self):
       # Return the total number of samples in the dataset
       return len(self.image_paths)

   def __getitem__(self, idx):
       # Get the head and tail of the image path
           
       train_img = readNifti_img(self.image_paths[idx])
       #print(train_img.shape)
       #print(train_img.dtype)
       train_mask = readNifti_mask(self.mask_paths[idx])
       #print(train_mask.shape)
       #print(train_mask.dtype)
       # subject = [train_img, train_mask, self.image_paths[idx]]

       subject = {'train_img': train_img,
                    'train_mask': train_mask,
                    'name':self.image_paths[idx]}
       
       #Add name here to check
       
       if self.transform:
           subject = self.transform(subject)

       # Return the subject dictionary
       return subject


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/rds/general/user/hhs21/home/anaconda3/envs/pyseghpc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/tmp/pbs.9223122.pbs/ipykernel_2785014/2927388279.py", line 2, in <module>
    from torch.utils.data import Dataset
ModuleNotFoundError: No module named 'torch'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/rds/general/user/hhs21/home/anaconda3/envs/pyseghpc/lib/python3.11/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/rds/general/user/hhs21/home/anaconda3/envs/pyseghpc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/rds/general/user/hhs21/home/anaconda3/envs/pyseghpc/lib/pyt

In [3]:
def main():
    return readNifti_mask("C:/Users/Haziq/OneDrive - Imperial College London/Third Year/Fetal VR Group Project/Files/3D 112 Dataset/test/mask/Case_122_Pre/Case_122_Pre_mask_time002.nii")

def voltosl(vol,sliceno):
    #Volume is shape [1,1,112,112,112]
    #Need to plot each slice at sliceno
    #vol = vol[2,:,:,20].cpu()
    vol=torch.argmax(vol,0).cpu()
    vol = vol[:,:,20]
    # print(vol)
    slice = np.asarray(vol.detach().numpy())
    slice = slice.reshape([112,112])
    #print(slice.shape)
    return slice
    print(y.shape)

if __name__ == "__main__":
    #import matplotlib.pyplot as plt
    #plt.imshow(voltosl(main(),40))
    pass